# AlexNet on Fashion-MNIST: Activation Function Comparison

In [ ]:

import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import pandas as pd


In [ ]:

# Load Fashion-MNIST dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
x_train = x_train[..., np.newaxis]
x_test = x_test[..., np.newaxis]
print(f"Training data shape: {x_train.shape}, Test data shape: {x_test.shape}")


In [ ]:

# Resize and normalize
resize_and_rescale = tf.keras.Sequential([
    layers.Resizing(224, 224),
    layers.Rescaling(1./255)
])

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_ds = train_ds.map(lambda x, y: (resize_and_rescale(x), y))
train_ds = train_ds.shuffle(buffer_size=10000).batch(64)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_ds = test_ds.map(lambda x, y: (resize_and_rescale(x), y))
test_ds = test_ds.batch(64)


In [ ]:

# Build AlexNet model with variable activation
def build_alexnet(activation):
    model = models.Sequential([
        layers.Conv2D(96, (11,11), strides=4, activation=activation, input_shape=(224,224,1)),
        layers.BatchNormalization(),
        layers.MaxPooling2D(pool_size=(3,3), strides=2),

        layers.Conv2D(256, (5,5), padding='same', activation=activation),
        layers.BatchNormalization(),
        layers.MaxPooling2D(pool_size=(3,3), strides=2),

        layers.Conv2D(384, (3,3), padding='same', activation=activation),
        layers.Conv2D(384, (3,3), padding='same', activation=activation),
        layers.Conv2D(256, (3,3), padding='same', activation=activation),
        layers.MaxPooling2D(pool_size=(3,3), strides=2),

        layers.Flatten(),
        layers.Dense(4096, activation=activation),
        layers.Dropout(0.5),
        layers.Dense(4096, activation=activation),
        layers.Dropout(0.5),
        layers.Dense(10, activation='softmax')
    ])
    return model


In [ ]:

# Define training function
def train_and_evaluate(activation_name, activation_fn):
    print(f"Training with {activation_name} activation")
    model = build_alexnet(activation_fn)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    history = model.fit(train_ds, epochs=5, validation_data=test_ds)
    test_loss, test_acc = model.evaluate(test_ds)
    print(f"{activation_name} Test Accuracy: {test_acc:.4f}")
    return activation_name, test_acc


In [ ]:

# Train with ReLU, Tanh, LeakyReLU
results = []

# ReLU
results.append(train_and_evaluate("ReLU", "relu"))

# Tanh
results.append(train_and_evaluate("Tanh", "tanh"))

# LeakyReLU: define a wrapper function
from tensorflow.keras.layers import LeakyReLU

def build_alexnet_leakyrelu():
    model = models.Sequential([
        layers.Conv2D(96, (11,11), strides=4, input_shape=(224,224,1)),
        LeakyReLU(alpha=0.01),
        layers.BatchNormalization(),
        layers.MaxPooling2D(pool_size=(3,3), strides=2),

        layers.Conv2D(256, (5,5), padding='same'),
        LeakyReLU(alpha=0.01),
        layers.BatchNormalization(),
        layers.MaxPooling2D(pool_size=(3,3), strides=2),

        layers.Conv2D(384, (3,3), padding='same'),
        LeakyReLU(alpha=0.01),
        layers.Conv2D(384, (3,3), padding='same'),
        LeakyReLU(alpha=0.01),
        layers.Conv2D(256, (3,3), padding='same'),
        LeakyReLU(alpha=0.01),
        layers.MaxPooling2D(pool_size=(3,3), strides=2),

        layers.Flatten(),
        layers.Dense(4096),
        LeakyReLU(alpha=0.01),
        layers.Dropout(0.5),
        layers.Dense(4096),
        LeakyReLU(alpha=0.01),
        layers.Dropout(0.5),
        layers.Dense(10, activation='softmax')
    ])
    return model

# Train LeakyReLU model
print("Training with LeakyReLU activation")
model = build_alexnet_leakyrelu()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
history = model.fit(train_ds, epochs=5, validation_data=test_ds)
test_loss, test_acc = model.evaluate(test_ds)
print(f"LeakyReLU Test Accuracy: {test_acc:.4f}")
results.append(("LeakyReLU", test_acc))


In [ ]:

# Final Comparison Table
df = pd.DataFrame(results, columns=["Activation Function", "Test Accuracy"])
print(df)
